<a href="https://colab.research.google.com/github/dmswl0707/CoronaCoprosperityCard_DataScience/blob/main/Card_Payment_Ratio_by_Age.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import json

from google.colab import drive
drive.mount('/gdrive') #구글 드라이브에 접근


accidents=pd.read_csv('/gdrive/My Drive/colab Notebooks/data/card_2020.csv') #구글 드라이브에서 파일 읽어오기
accidents
